# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [58]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
#from api_keys import geoapify_key

# Geoapify API Key
geoapify_key = "d985501efebf41ca9c659c7b6f782493"

In [59]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ballymena,54.8636,-6.2763,282.05,92,100,3.97,GB,2023-03-23
1,1,Marzuq,14.4000,46.4667,296.56,65,10,2.63,YE,2023-03-23
2,2,Nikolskoye,59.7035,30.7861,278.66,95,66,3.46,RU,2023-03-23
3,3,Sola,-13.8833,167.5500,300.10,74,97,3.01,VU,2023-03-23
4,4,Kodiak,57.7900,-152.4072,271.88,43,0,6.69,US,2023-03-23


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [77]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

map_plot_1

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [119]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[(city_data_df['Max Temp']<300) & 
                                  (city_data_df['Max Temp']>250) & 
                                  (city_data_df['Wind Speed']<10)& 
                                  (city_data_df['Humidity']<50)]
                                                                   
# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df.count()

City_ID       46
City          46
Lat           46
Lng           46
Max Temp      46
Humidity      46
Cloudiness    46
Wind Speed    46
Country       46
Date          46
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [134]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(ideal_city_data_df, columns=['City','Country','Lat','Lng','Humidity'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,Kodiak,US,57.7900,-152.4072,43,
13,Saint Anthony,US,45.0205,-93.2180,46,
27,Pa Sang,TH,18.5262,98.9394,46,
71,Caborca,MX,30.5838,-111.1010,35,
79,Korla,CN,41.7597,86.1469,29,
112,Yulara,AU,-25.2406,130.9889,32,
133,Silver City,US,32.7701,-108.2803,45,
143,Zaragoza,ES,41.6561,-0.8773,30,
152,Constitucion,CL,-35.3333,-72.4167,47,
153,Salimpur,IN,28.1280,76.2766,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [135]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories": "accommodations.hotel",
    "limit": 10,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Kodiak - nearest hotel: No hotel found
Saint Anthony - nearest hotel: No hotel found
Pa Sang - nearest hotel: No hotel found
Caborca - nearest hotel: No hotel found
Korla - nearest hotel: No hotel found
Yulara - nearest hotel: No hotel found
Silver City - nearest hotel: No hotel found
Zaragoza - nearest hotel: No hotel found
Constitucion - nearest hotel: No hotel found
Salimpur - nearest hotel: No hotel found
Aswan - nearest hotel: No hotel found
Durango - nearest hotel: No hotel found
Dabat - nearest hotel: No hotel found
Comodoro Rivadavia - nearest hotel: No hotel found
Buraydah - nearest hotel: No hotel found
Copiapo - nearest hotel: No hotel found
Mandalgovi - nearest hotel: No hotel found
Mecca - nearest hotel: No hotel found
Sayyan - nearest hotel: No hotel found
Superior - nearest hotel: No hotel found
Jiuquan - nearest hotel: No hotel found
Asyut - nearest hotel: No hotel found
Zhangye - nearest hotel: No hotel found
Uglekamensk - nearest hotel: No hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
4,Kodiak,US,57.7900,-152.4072,43,No hotel found
13,Saint Anthony,US,45.0205,-93.2180,46,No hotel found
27,Pa Sang,TH,18.5262,98.9394,46,No hotel found
71,Caborca,MX,30.5838,-111.1010,35,No hotel found
79,Korla,CN,41.7597,86.1469,29,No hotel found
112,Yulara,AU,-25.2406,130.9889,32,No hotel found
133,Silver City,US,32.7701,-108.2803,45,No hotel found
143,Zaragoza,ES,41.6561,-0.8773,30,No hotel found
152,Constitucion,CL,-35.3333,-72.4167,47,No hotel found
153,Salimpur,IN,28.1280,76.2766,46,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [136]:
%%capture --no-display

# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot_3

# Display the map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)